# Preprocessing one go

In [ ]:
os.system("rm -rf data/*")

for subset in itertools.combinations([i for i in range(10)],2):

    x_train_subset = x_train[(y_train == subset[0]) | (y_train == subset[1])]
    y_train_subset = y_train[(y_train == subset[0]) | (y_train == subset[1])]
    y_train_subset_binary = np.where(y_train_subset == subset[0], 0, y_train_subset)
    y_train_subset_binary = np.where(y_train_subset_binary == subset[1], 1, y_train_subset_binary)

    x_test_subset = x_test[(y_test == subset[0]) | (y_test == subset[1])]
    y_test_subset = y_test[(y_test == subset[0]) | (y_test == subset[1])]
    y_test_subset_binary = np.where(y_test_subset == subset[0], 0, y_test_subset)
    y_test_subset_binary = np.where(y_test_subset_binary == subset[1], 1, y_test_subset_binary)

    x_train_subset = x_train_subset.reshape(x_train_subset.shape[0],784)
    x_test_subset = x_test_subset.reshape(x_test_subset.shape[0],784)

    for preprocessing_method, preprocessing_name in pre.preprocessing_methods():

        if not os.path.isdir("data/{}".format(preprocessing_name)):
            print("making directory: data/{}".format(preprocessing_name))
            os.mkdir("data/{}".format(preprocessing_name))

        x_train_pre, x_test_pre = preprocessing_method(x_train_subset, x_test_subset,outputsize=4)
        #normalize again
        x_train_pre = pre.minmax_scaler(x_train_pre,min=0,max=1)
        x_test_pre = pre.minmax_scaler(x_test_pre,min=0,max=1)

        if np.any(np.isnan(x_train_pre)) or np.any(np.isnan(x_test_pre)):
            print("NaN detected",subset,preprocessing_name)

        subset_name = str(subset).replace("(","").replace(")","").replace(" ","")

        if not os.path.isdir("data/{}/{}".format(preprocessing_name,subset_name)):
            print("making directory: data/{}/{}".format(preprocessing_name,subset_name))
            os.mkdir("data/{}/{}".format(preprocessing_name,subset_name))

        for type, dataset in zip(["x_train","x_test","y_train","y_test","y_train_binary","y_test_binary"],[x_train_pre,x_test_pre,y_train_subset,y_test_subset,y_train_subset_binary,y_test_subset_binary]):
            np.save("data/"+preprocessing_name+"/"+str(subset_name)+"/"+type+".npy",dataset)